In [1]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf

import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']

for intent in intents['intents']:
    try:
        for pattern in intent['patterns']:
            wordList = nltk.word_tokenize(pattern)
            words.extend(wordList)
            documents.append((wordList, intent['tag']))
            if intent['tag'] not in classes:
                classes.append(intent['tag'])
    except Exception as ex:
        print(intent, ">>", ex)

words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
outputEmpty = [0] * len(classes)

for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):]


model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(trainX), np.array(trainY), epochs=200, batch_size=5, verbose=1)
model.save('bourntec_chatbot_model.h5', hist)
print('Done')


Epoch 1/200
132/132 [==============================] - 4s 5ms/step - loss: 3.3833 - accuracy: 0.1094
Epoch 2/200
132/132 [==============================] - 1s 4ms/step - loss: 2.9461 - accuracy: 0.2204
Epoch 3/200
132/132 [==============================] - 1s 4ms/step - loss: 2.5711 - accuracy: 0.3040
Epoch 4/200
132/132 [==============================] - 1s 4ms/step - loss: 2.2697 - accuracy: 0.3587
Epoch 5/200
132/132 [==============================] - 1s 4ms/step - loss: 2.0874 - accuracy: 0.4012
Epoch 6/200
132/132 [==============================] - 1s 4ms/step - loss: 1.8203 - accuracy: 0.4878
Epoch 7/200
132/132 [==============================] - 1s 4ms/step - loss: 1.7175 - accuracy: 0.5152
Epoch 8/200
132/132 [==============================] - 1s 4ms/step - loss: 1.6502 - accuracy: 0.5258
Epoch 9/200
132/132 [==============================] - 1s 5ms/step - loss: 1.5156 - accuracy: 0.5623
Epoch 10/200
132/132 [==============================] - 1s 4ms/step - loss: 1.4098 - accura

132/132 [==============================] - 1s 5ms/step - loss: 0.4558 - accuracy: 0.8632
Epoch 82/200
132/132 [==============================] - 1s 4ms/step - loss: 0.6171 - accuracy: 0.8222
Epoch 83/200
132/132 [==============================] - 1s 4ms/step - loss: 0.6190 - accuracy: 0.8419
Epoch 84/200
132/132 [==============================] - 1s 4ms/step - loss: 0.6556 - accuracy: 0.8100
Epoch 85/200
132/132 [==============================] - 1s 4ms/step - loss: 0.5627 - accuracy: 0.8359
Epoch 86/200
132/132 [==============================] - 1s 5ms/step - loss: 0.5343 - accuracy: 0.8450
Epoch 87/200
132/132 [==============================] - 1s 4ms/step - loss: 0.5408 - accuracy: 0.8511
Epoch 88/200
132/132 [==============================] - 1s 4ms/step - loss: 0.4715 - accuracy: 0.8526
Epoch 89/200
132/132 [==============================] - 1s 5ms/step - loss: 0.5314 - accuracy: 0.8511
Epoch 90/200
132/132 [==============================] - 1s 4ms/step - loss: 0.5389 - accuracy: 

132/132 [==============================] - 1s 4ms/step - loss: 0.4203 - accuracy: 0.8860
Epoch 161/200
132/132 [==============================] - 1s 4ms/step - loss: 0.4274 - accuracy: 0.8647
Epoch 162/200
132/132 [==============================] - 1s 4ms/step - loss: 0.5152 - accuracy: 0.8526
Epoch 163/200
132/132 [==============================] - 1s 4ms/step - loss: 0.5587 - accuracy: 0.8480
Epoch 164/200
132/132 [==============================] - 1s 4ms/step - loss: 0.4489 - accuracy: 0.8511
Epoch 165/200
132/132 [==============================] - 1s 4ms/step - loss: 0.4086 - accuracy: 0.8678
Epoch 166/200
132/132 [==============================] - 1s 4ms/step - loss: 0.3641 - accuracy: 0.8845
Epoch 167/200
132/132 [==============================] - 1s 4ms/step - loss: 0.4512 - accuracy: 0.8571
Epoch 168/200
132/132 [==============================] - 1s 4ms/step - loss: 0.4310 - accuracy: 0.8632
Epoch 169/200
132/132 [==============================] - 1s 4ms/step - loss: 0.3944 - a

D:\Anaconda\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Done
